In [12]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import vgg16

from pathlib import Path
import joblib


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'


male_path = Path("../Data/Dogs/train/male")
female_path = Path("../Data/Dogs/train/female")

In [13]:
train_images = []
train_target = []

height = 128
width = 128

for current_file in male_path.glob("*.jpg"):
    current_img = image.load_img(current_file, target_size=(height,width))
    current_img = image.img_to_array(current_img)  
    train_images.append(current_img)
    train_target.append(0)

In [14]:
for current_file in female_path.glob("*.jpg"):
    current_img = image.load_img(current_file, target_size=(height,width))
    current_img = image.img_to_array(current_img)  
    train_images.append(current_img)
    train_target.append(1)

In [15]:
x_train = np.array(train_images)
y_train = np.array(train_target)


x_train = vgg16.preprocess_input(x_train) 

In [16]:
pretrained_model= vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(height,width,3)) 

In [17]:
features =  pretrained_model.predict(x_train) 

In [18]:
model = keras.Sequential()

model.add(keras.layers.Flatten(input_shape=features.shape[1:]))
 
model.add(keras.layers.Dense(256, activation='relu'))
 
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [21]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
 
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
result = model.fit(features, y_train,\
                   epochs=200,\
                   validation_data=[features, y_train],\
                   callbacks=[early_stopping])

Train on 41 samples, validate on 41 samples
Epoch 1/200
41/41 [==============================] - 0s 10ms/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 2/200
41/41 [==============================] - 0s 807us/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 3/200
41/41 [==============================] - 0s 854us/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 4/200
41/41 [==============================] - 0s 999us/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 5/200
41/41 [==============================] - 0s 901us/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 6/200
41/41 [==============================] - 0s 903us/sample - loss: 7.0794e-37 - accuracy: 1.0000 - val_loss: 7.0794e-37 - val_accuracy: 1.0000
Epoch 7/200
41/41 [====================

In [22]:
 
model.save("../Models/vgg_model.h5")
 